<a href="https://colab.research.google.com/github/lokenderquadrigo/CurrencyConverter/blob/main/Lab_7_Build_a_Contextual_Retrieval_based_RAG_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Contextual Retrieval based RAG System

## Install OpenAI, and LangChain dependencies

In [ ]:
%%capture
!pip install langchain #==0.3.10
!pip install langchain-openai #==0.2.12
!pip install langchain-community #==0.3.11

In [ ]:
%%capture
!pip install jq #==1.8.0

In [ ]:
%%capture
!pip install pymupdf #==1.25.1

## Install Chroma Vector DB and LangChain wrapper

In [ ]:
%%capture
!pip install langchain-chroma #==0.1.4

## Enter Open AI API Key

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [ ]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

ImportError: cannot import name 'LangSmithParams' from 'langchain_core.language_models.chat_models' (/usr/local/lib/python3.12/dist-packages/langchain_core/language_models/chat_models.py)

## Loading and Processing the Data

### Get the dataset

In [ ]:
# if you can't download using the following code
# go to https://drive.google.com/file/d/1aZxZejfteVuofISodUrY2CDoyuPLYDGZ download it
# manually upload it on colab
!gdown 1aZxZejfteVuofISodUrY2CDoyuPLYDGZ

In [ ]:
!unzip rag_docs.zip

### Load and Process JSON Documents

In [ ]:
from langchain.document_loaders import JSONLoader

loader = JSONLoader(file_path='./rag_docs/wikidata_rag_demo.jsonl',
                    jq_schema='.',
                    text_content=False,
                    json_lines=True)
wiki_docs = loader.load()

In [ ]:
len(wiki_docs)

In [ ]:
wiki_docs[3]

In [ ]:
import json
from langchain.docstore.document import Document
wiki_docs_processed = []

for doc in wiki_docs:
    doc = json.loads(doc.page_content)
    metadata = {
        "title": doc['title'],
        "id": doc['id'],
        "source": "Wikipedia",
        "page": 1
    }
    data = ' '.join(doc['paragraphs'])
    wiki_docs_processed.append(Document(page_content=data, metadata=metadata))

In [ ]:
wiki_docs_processed[3]

### Load and Process PDF documents

#### Create Chunk Contexts for Contextual Retrieval

![](https://i.imgur.com/LRhKHzk.png)

In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
# Import necessary libraries from the LangChain framework and standard libraries
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document


# --- Helper Function to Generate Context for a Single Chunk ---

def generate_chunk_context(document, chunk):
    """
    Generates a brief, contextual summary for a text chunk based on the entire document.

    Args:
        document (str): The full content of the original document.
        chunk (str): The specific text chunk that needs a contextual summary.

    Returns:
        str: An AI-generated contextual summary for the chunk.
    """

    # This is the prompt template that instructs the language model.
    # It clearly defines the role of the AI, the inputs (the full paper and the specific chunk),
    # and the desired output format (a concise context).
    chunk_process_prompt = """You are an AI assistant specializing in research paper analysis.
                            Your task is to provide brief, relevant context for a chunk of text
                            based on the following research paper.

                            Here is the research paper:
                            <paper>
                            {paper}
                            </paper>

                            Here is the chunk we want to situate within the whole document:
                            <chunk>
                            {chunk}
                            </chunk>

                            Provide a concise context (3-4 sentences max) for this chunk,
                            considering the following guidelines:

                            - Give a short succinct context to situate this chunk within the overall document
                            for the purposes of improving search retrieval of the chunk.
                            - Answer only with the succinct context and nothing else.
                            - Context should be mentioned like 'Focuses on ....'
                            do not mention 'this chunk or section focuses on...'

                            Context:
                         """

    # Create a prompt object from the string template
    prompt_template = ChatPromptTemplate.from_template(chunk_process_prompt)

    # Define the "chain" using LangChain Expression Language (LCEL).
    # 1. The prompt_template takes the 'paper' and 'chunk' inputs.
    # 2. The formatted prompt is passed to the language model ('chatgpt').
    # 3. The model's output is parsed into a simple string by 'StrOutputParser'.
    agentic_chunk_chain = (prompt_template
                           |
                           chatgpt # Your language model instance
                           |
                           StrOutputParser())

    # Execute the chain by providing the actual document and chunk content.
    context = agentic_chunk_chain.invoke({'paper': document, 'chunk': chunk})

    # Return the generated context string.
    return context

In [ ]:

from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid # Used for generating unique IDs for each chunk

# --- Main Function to Create Contextual Chunks from a File ---

def create_contextual_chunks(file_path, chunk_size=3500, chunk_overlap=0):
    """
    Loads a PDF, splits it into chunks, and adds a contextual summary to each chunk.

    Args:
        file_path (str): The path to the PDF file.
        chunk_size (int): The maximum number of characters for each chunk.
        chunk_overlap (int): The number of characters to overlap between consecutive chunks.

    Returns:
        list[Document]: A list of LangChain Document objects, where each document's
                        page_content is the context + original chunk text.
    """

    print('Loading pages:', file_path)
    # Initialize the PDF loader with the file path
    loader = PyMuPDFLoader(file_path)
    # Load the document into a list of LangChain Document objects, one per page
    doc_pages = loader.load()

    print('Chunking pages:', file_path)
    # Initialize the text splitter with the desired chunk size and overlap
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                              chunk_overlap=chunk_overlap)
    # Split the loaded pages into smaller chunks
    doc_chunks = splitter.split_documents(doc_pages)

    print('Generating contextual chunks:', file_path)
    # Reassemble the entire document's text to provide full context for each chunk
    original_doc = '\n'.join([doc.page_content for doc in doc_chunks])

    # This list will store the final, enriched chunks
    contextual_chunks = []

    # Loop through each raw chunk to generate its context
    for chunk in doc_chunks:
        # Get the text content and metadata from the current chunk
        chunk_content = chunk.page_content
        chunk_metadata = chunk.metadata

        # Create a new, updated metadata dictionary for better organization
        chunk_metadata_upd = {
            'id': str(uuid.uuid4()),  # Add a unique identifier
            'page': chunk_metadata['page'],  # Keep the original page number
            'source': chunk_metadata['source'],  # Keep the original file path
            'title': chunk_metadata['source'].split('/')[-1] # Extract just the filename as title
        }

        # Call our helper function to generate the context for this specific chunk
        context = generate_chunk_context(original_doc, chunk_content)

        # Create a new LangChain Document object
        # The content is the generated context, a newline, and then the original chunk's content
        # The metadata is our newly created dictionary
        contextual_chunks.append(Document(page_content=context + '\n' + chunk_content,
                                          metadata=chunk_metadata_upd))

    print('Finished processing:', file_path)
    print()

    # Return the list of newly created, context-aware chunks
    return contextual_chunks

In [ ]:
from glob import glob

pdf_files = glob('./rag_docs/*.pdf')
pdf_files

In [ ]:
paper_docs = []
for fp in pdf_files:
    paper_docs.extend(create_contextual_chunks(file_path=fp, chunk_size=3500))

In [ ]:
len(paper_docs)

In [ ]:
paper_docs[0]

In [ ]:
display(Markdown(paper_docs[10].page_content))

### Combine all document chunks in one list

In [ ]:
len(wiki_docs_processed)

In [ ]:
total_docs = wiki_docs_processed + paper_docs
len(total_docs)

## Index Document Chunks and Embeddings in Vector DB

Here we initialize a connection to a Chroma vector DB client, and also we want to save to disk, so we simply initialize the Chroma client and pass the directory where we want the data to be saved to.

In [ ]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes < 30s on Colab
chroma_db = Chroma.from_documents(documents=total_docs,
                                  collection_name='my_context_db',
                                  embedding=openai_embed_model,
                                  # need to set the distance function to cosine else it uses euclidean by default
                                  # check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./my_context_db")

### Load Vector DB from disk

This is just to show once you have a vector database on disk you can just load and create a connection to it anytime

In [ ]:
# load from disk
chroma_db = Chroma(persist_directory="./my_context_db",
                   collection_name='my_context_db',
                   embedding_function=openai_embed_model)

In [ ]:
chroma_db

### Semantic Similarity based Retrieval

We use simple cosine similarity here and retrieve the top 5 similar documents based on the user input query

In [ ]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 5})

In [ ]:
from IPython.display import display, Markdown

def display_docs(docs):
    for doc in docs:
        print('Metadata:', doc.metadata)
        print('Content Brief:')
        display(Markdown(doc.page_content[:1000]))
        print()

In [ ]:
query = "what is machine learning?"
top_docs = similarity_retriever.invoke(query)
display_docs(top_docs)

In [ ]:
query = "what is the difference between transformers and vision transformers?"
top_docs = similarity_retriever.invoke(query)
display_docs(top_docs)

## Build the RAG Pipeline

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer detailed and well formatted based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (similarity_retriever
                      |
                    format_docs),
        "question": RunnablePassthrough()
    }
      |
    rag_prompt_template
      |
    chatgpt
)

In [ ]:
from IPython.display import display, Markdown

query = "What is machine learning?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

In [ ]:
query = "What is a CNN?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

In [ ]:
query = "How is a resnet better than a CNN?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

In [ ]:
query = "What is NLP and its relation to linguistics?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

In [ ]:
query = "What is the difference between AI, ML and DL?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

In [ ]:
query = "What is the difference between transformers and vision transformers?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

In [ ]:
query = "How is self-attention important in transformers?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

In [ ]:
query = "How does a resnet work?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

In [ ]:
query = "What is LangGraph?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

In [ ]:
query = "What is an Agentic AI System?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

In [ ]:
query = "What is LangChain?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

# Build a RAG System with Sources

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer detailed and well formatted based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)

In [ ]:
# Import necessary components from LangChain
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from operator import itemgetter # A standard Python function to get an item from a dictionary

# --- Initialization ---
# Initialize the language model instance. We'll use this for generation.
# "gpt-4o-mini" is a fast and capable model. temperature=0 makes its output more deterministic.
chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# NOTE: The following variables are assumed to be defined elsewhere in your code:
# - similarity_retriever: A LangChain retriever object that fetches documents based on a query.
# - rag_prompt_template: A ChatPromptTemplate that takes "context" and "question" as input.


# --- Helper Function ---
def format_docs(docs):
    """A helper function to concatenate the page_content of multiple documents into a single string."""
    return "\n\n".join(doc.page_content for doc in docs)


# --- Chain 1: The Core RAG Generation Chain ---
# This chain is responsible for generating the final answer once it has the context and question.
src_rag_response_chain = (
    # This dictionary defines the inputs for the prompt template.
    {
        # The "context" key's value is processed in two steps:
        # 1. itemgetter('context'): Extracts the list of documents from the input dictionary.
        # 2. RunnableLambda(format_docs): Applies our helper function to format the documents into a single string.
        "context": (itemgetter('context') | RunnableLambda(format_docs)),

        # The "question" key's value is simply extracted from the input dictionary.
        "question": itemgetter("question")
    }
    |
    # The dictionary with formatted context and question is passed to the prompt template.
    rag_prompt_template
    |
    # The formatted prompt is passed to the language model.
    chatgpt
    |
    # The output from the model is parsed into a clean string.
    StrOutputParser()
)


# --- Chain 2: The Full RAG Pipeline that includes Sources ---
# This is the main chain that orchestrates the entire process, from question to final output.
rag_chain_w_sources = (
    # This initial step performs retrieval. The input to this whole chain is the user's question.
    {
        # The retriever is called with the user's question to fetch relevant documents.
        # The result is assigned to the "context" key.
        "context": similarity_retriever,

        # RunnablePassthrough() simply passes the original user question through to the "question" key.
        # The output of this dictionary is: {"context": [docs], "question": "user's question"}
        "question": RunnablePassthrough()
    }
    |
    # RunnablePassthrough.assign() is a key component. It takes the dictionary from the previous step
    # and adds a new key to it. Here, we are adding a key named "response".
    # The value of "response" is generated by running the `src_rag_response_chain` with the
    # context and question from the previous step.
    RunnablePassthrough.assign(response=src_rag_response_chain)
)

# Final Output Structure when invoking `rag_chain_w_sources`:
# {
#   'context': [Document(page_content='...'), ...],  <- The source documents
#   'question': 'What is the capital of France?',     <- The original question
#   'response': 'The capital of France is Paris.'     <- The LLM's answer
# }

In [ ]:
query = "What is machine learning?"
result = rag_chain_w_sources.invoke(query)
result

In [ ]:
from IPython.display import display, Markdown

def display_results(result_obj):
    print('Query:')
    display(Markdown(result_obj['question']))
    print()
    print('Response:')
    display(Markdown(result_obj['response']))
    print('='*50)
    print('Sources:')
    for source in result_obj['context']:
        print('Metadata:', source.metadata)
        print('Content Brief:')
        display(Markdown(source.page_content))
        print()


In [ ]:
query = "What is machine learning?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

In [ ]:
query = "What is the difference between AI, ML and DL?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

In [ ]:
query = "What is the difference between transformers and vision transformers?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

In [ ]:
query = "What is an Agentic AI System?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

# Build a RAG System with Source Citations Agentic Pipeline

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer detailed and well formatted based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)
rag_prompt_template.pretty_print()

In [ ]:
citations_prompt = """You are an assistant who is an expert in analyzing answers to questions
                      and finding out referenced citations from context articles.

                      Given the following question, context and generated answer,
                      analyze the generated answer and quote citations from context articles
                      that can be used to justify the generated answer.

                      Question:
                      {question}

                      Context Articles:
                      {context}

                      Answer:
                      {answer}
                  """

cite_prompt_template = ChatPromptTemplate.from_template(citations_prompt)
cite_prompt_template.pretty_print()

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class Citation(BaseModel):
    id: str = Field(description="""The string ID of a SPECIFIC context article
                                   which justifies the answer.""")
    source: str = Field(description="""The source of the SPECIFIC context article
                                       which justifies the answer.""")
    title: str = Field(description="""The title of the SPECIFIC context article
                                      which justifies the answer.""")
    page: int = Field(description="""The page number of the SPECIFIC context article
                                     which justifies the answer.""")
    quotes: str = Field(description="""The VERBATIM sentences from the SPECIFIC context article
                                      that are used to generate the answer.
                                      Should be exact sentences from context article without missing words.""")


class QuotedCitations(BaseModel):
    """Quote citations from given context articles
       that can be used to justify the generated answer. Can be multiple articles."""
    citations: List[Citation] = Field(description="""Citations (can be multiple) from the given
                                                     context articles that justify the answer.""")

In [ ]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from operator import itemgetter


# --- Model Initialization ---

# Standard language model for generating plain text answers.
chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# A specialized version of the model that is forced to output JSON matching the 'QuotedCitations' schema.
# This is the key to getting reliable, structured citation data.
structured_chatgpt = chatgpt.with_structured_output(QuotedCitations)

# NOTE: These variables are assumed to be defined elsewhere:
# - similarity_retriever: A retriever that fetches relevant documents.
# - rag_prompt_template: A prompt for generating the initial answer.
# - cite_prompt_template: A prompt specifically for generating citations based on an answer.


# --- Helper Function ---

def format_docs_with_metadata(docs: List[Document]) -> str:
    """
    Formats documents to include their metadata explicitly in the context string.
    This helps the LLM to easily access information needed for citations.
    """
    formatted_docs = [
        f"""Context Article ID: {doc.metadata['id']}
            Context Article Source: {doc.metadata['source']}
            Context Article Title: {doc.metadata['title']}
            Context Article Page: {doc.metadata['page']}
            Context Article Details: {doc.page_content}
         """
        for i, doc in enumerate(docs)
    ]
    # Join all formatted doc strings into a single block of text.
    return "\n\n" + "\n\n".join(formatted_docs)


# --- Chain 1: Generates the initial text answer ---

rag_response_chain = (
    {
        # Format the retrieved documents using our detailed helper function.
        "context": (itemgetter('context') | RunnableLambda(format_docs_with_metadata)),
        # Pass the original question through.
        "question": itemgetter("question")
    }
    |
    # Use the standard RAG prompt.
    rag_prompt_template
    |
    # Use the standard chat model to generate a text answer.
    chatgpt
    |
    # Parse the output into a string.
    StrOutputParser()
)


# --- Chain 2: Generates structured citations for the answer ---

cite_response_chain = (
    {
        # Pass the original, unformatted context through.
        "context": itemgetter('context'),
        # Pass the original question through.
        "question": itemgetter("question"),
        # IMPORTANT: This chain requires the 'answer' generated by the previous chain.
        "answer": itemgetter("answer")
    }
    |
    # Use the specialized citation prompt.
    cite_prompt_template
    |
    # Use the structured output model to get a JSON object of citations.
    structured_chatgpt
)


# --- Chain 3: The final orchestrator chain ---

rag_chain_w_citations = (
    # Step 1: Retrieval. Fetch context documents based on the question.
    # Output of this step: {"context": [docs], "question": "user_question"}
    {
        "context": similarity_retriever,
        "question": RunnablePassthrough()
    }
    |
    # Step 2: Generate Answer. Run the first chain to get the text answer and
    # add it to the dictionary under the key "answer".
    # Output of this step: {"context": [docs], "question": "user_question", "answer": "text_answer"}
    RunnablePassthrough.assign(answer=rag_response_chain)
    |
    # Step 3: Generate Citations. Run the second chain using the output from the previous step.
    # The result (structured citations) is added to the dictionary under the key "citations".
    # Output of this step: {"context": ..., "question": ..., "answer": ..., "citations": ...}
    RunnablePassthrough.assign(citations=cite_response_chain)
)

In [ ]:
query = "What is machine learning"
result = rag_chain_w_citations.invoke(query)
result

In [ ]:
result['citations'].dict()['citations']

In [ ]:
import re
# used mostly for nice display formatting, ignore if not needed
def get_cited_context(result_obj):
    # Dictionary to hold separate citation information for each unique source and title combination
    source_with_citations = {}

    def highlight_text(context, quote):
        # Normalize whitespace and remove unnecessary punctuation
        quote = re.sub(r'\s+', ' ', quote).strip()
        context = re.sub(r'\s+', ' ', context).strip()

        # Split quote into phrases, being careful with punctuation
        phrases = [phrase.strip() for phrase in re.split(r'[.!?]', quote) if phrase.strip()]

        highlighted_context = context

        for phrase in phrases: # for each quoted phrase

            # Create regex pattern to match cited phrases
            # Escape special regex characters, but preserve word boundaries
            escaped_phrase = re.escape(phrase)
            # Create regex pattern that allows for slight variations
            pattern = re.compile(r'\b' + escaped_phrase + r'\b', re.IGNORECASE)

            # Replace all matched phrases with bolded version
            highlighted_context = pattern.sub(lambda m: f"**{m.group(0)}**", highlighted_context)

        return highlighted_context

    # Process the citation data
    for cite in result_obj['citations'].dict()['citations']:
        cite_id = cite['id']
        title = cite['title']
        source = cite['source']
        page = cite['page']
        quote = cite['quotes']

        # Check if the (source, title) key exists, and initialize if it doesn't
        if (source, title) not in source_with_citations:
            source_with_citations[(source, title)] = {
                'title': title,
                'source': source,
                'citations': []
            }

        # Find or create the citation entry for this unique (id, page) combination
        citation_entry = next(
            (c for c in source_with_citations[(source, title)]['citations'] if c['id'] == cite_id and c['page'] == page),
            None
        )
        if citation_entry is None:
            citation_entry = {'id': cite_id, 'page': page, 'quote': [quote], 'context': None}
            source_with_citations[(source, title)]['citations'].append(citation_entry)
        else:
            citation_entry['quote'].append(quote)

    # Process context data
    for context in result_obj['context']:
        context_id = context.metadata['id']
        context_page = context.metadata['page']
        source = context.metadata['source']
        title = context.metadata['title']
        page_content = context.page_content

        # Match the context to the correct citation entry by source, title, id, and page
        if (source, title) in source_with_citations:
            for citation in source_with_citations[(source, title)]['citations']:
                if citation['id'] == context_id and citation['page'] == context_page:
                    # Apply highlighting for each quote in the citation's quote list
                    highlighted_content = page_content
                    for quote in citation['quote']:
                        highlighted_content = highlight_text(highlighted_content, quote)
                    citation['context'] = highlighted_content

    # Convert the dictionary to a list of dictionaries for separate entries
    final_result_list = [
        {
            'title': details['title'],
            'source': details['source'],
            'citations': details['citations']
        }
        for details in source_with_citations.values()
    ]

    return final_result_list


In [ ]:
get_cited_context(result)

In [ ]:
from IPython.display import display, Markdown

def display_results(result_obj):
    print('Query:')
    display(Markdown(result_obj['question']))
    print()
    print('Response:')
    display(Markdown(result_obj['answer']))
    print('='*50)
    print('Sources:')
    cited_context = get_cited_context(result_obj)
    for source in cited_context:
        print('Title:', source['title'], ' ', 'Source:', source['source'])
        print('Citations:')
        for citation in source['citations']:
            print('ID:', citation['id'], ' ', 'Page:', citation['page'])
            print('Cited Quotes:')
            display(Markdown('*'+' '.join(citation['quote'])+'*'))
            print('Cited Context:')
            display(Markdown(citation['context']))
            print()


In [ ]:
display_results(result)

In [ ]:
query = "What is AI, ML and DL?"
result = rag_chain_w_citations.invoke(query)
display_results(result)

In [ ]:
query = "How is Machine learning related to supervised learning and clustering?"
result = rag_chain_w_citations.invoke(query)
display_results(result)

In [ ]:
query = "What is the difference between transformers and vision transformers?"
result = rag_chain_w_citations.invoke(query)
display_results(result)